In [ ]:
%env OPENAI_API_KEY=<PUT_YOUR_API_KEY_HERE>

In [2]:
def earth_to_mars_time(earth_seconds):
    # Length of a Martian sol in Earth seconds
    mars_sol_length = 88775.244

    # Convert Earth seconds to Mars seconds
    mars_seconds = earth_seconds * mars_sol_length / 86400

    # Calculate Mars time components
    sols = int(mars_seconds // mars_sol_length)
    mars_seconds %= mars_sol_length
    hours = int(mars_seconds // 3600)
    mars_seconds %= 3600
    minutes = int(mars_seconds // 60)
    seconds = int(mars_seconds % 60)

    return sols, hours, minutes, seconds


# Example usage
earth_seconds = 60  # Replace with any number of Earth seconds
mars_time = earth_to_mars_time(earth_seconds)
print(
    f"{earth_seconds} Earth seconds is {mars_time[0]} Mars day(s), {mars_time[1]} hours, {mars_time[2]} minutes, {mars_time[3]} seconds on Mars.")


60 Earth seconds is 0 Mars day(s), 0 hours, 1 minutes, 1 seconds on Mars.


In [5]:
import unittest

class TestEarthToMarsTime(unittest.TestCase):
    def test_small_number_of_seconds(self):
        self.assertEqual(earth_to_mars_time(1), (0, 0, 0, 1))

    def test_one_minute(self):
        self.assertEqual(earth_to_mars_time(60), (0, 0, 1, 1))

    def test_one_hour(self):
        self.assertEqual(earth_to_mars_time(3600), (0, 1, 1, 38))

    def test_one_earth_day(self):
        self.assertEqual(earth_to_mars_time(86400), (1, 0, 0, 0))

    def test_large_number_of_seconds(self):
        self.assertEqual(earth_to_mars_time(90000), (1, 1, 1, 38))

# Run the unit tests
unittest.main(argv=['first-arg-is-ignored'], exit=False)


.....
----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK


In [6]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


def gpt35(prompt, model="gpt-3.5-turbo-instruct", temperature=0.4, max_tokens=2500,
          top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"]):
    response = client.completions.create(model=model,
                                         prompt=prompt,
                                         temperature=temperature,
                                         max_tokens=max_tokens,
                                         top_p=top_p,
                                         stop=stop)
    message = response.choices[0].text.strip()
    return message


code = """
def earth_to_mars_time(earth_seconds):
    # Length of a Martian sol in Earth seconds
    mars_sol_length = 88775.244

    # Convert Earth seconds to Mars seconds
    mars_seconds = earth_seconds * mars_sol_length / 86400

    # Calculate Mars time components
    sols = int(mars_seconds // mars_sol_length)
    mars_seconds %= mars_sol_length
    hours = int(mars_seconds // 3600)
    mars_seconds %= 3600
    minutes = int(mars_seconds // 60)
    seconds = int(mars_seconds % 60)

    return sols, hours, minutes, seconds
"""


def explain_code(function_to_test, unit_test_package="unittest"):
    prompt = f""""# How to write great unit tests with {unit_test_package}

In this advanced tutorial for experts, we'll use Python 3.10 and `{unit_test_package}` to write a suite of unit tests to verify the behavior of the following function.
```python
{function_to_test}


Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
- First,"""
    response = gpt35(prompt)
    return response, prompt


code_explaination, prompt_to_explain_code = explain_code(code)
print(code_explaination)


we define a variable `mars_sol_length` which represents the length of a Martian sol (day) in Earth seconds. This value is based on the length of a Martian day as measured by NASA's Mars Exploration Rover, Opportunity.
- Next, we convert the input `earth_seconds` into Mars seconds by multiplying it by the ratio of the Martian day to the Earth day.
- Then, we calculate the number of full Martian sols (days) by dividing the total number of Mars seconds by the length of a Martian sol and rounding down to the nearest integer.
- We also calculate the remaining Mars seconds after accounting for the full sols.
- Finally, we use basic division and modulus operations to calculate the hours, minutes, and seconds components of the Mars time.


In [7]:
def generate_a_test_plan(full_code_explaination, unit_test_package="unittest"):
    prompt_to_explain_a_plan = f"""

A good unit test suite should aim to:
- Test the function's behavior for a wide range of possible inputs
- Test edge cases that the author may not have foreseen
- Take advantage of the features of `{unit_test_package}` to make the tests easy to write and maintain
- Be easy to read and understand, with clean code and descriptive names
- Be deterministic, so that the tests always pass or fail in the same way

`{unit_test_package}` has many convenient features that make it easy to write and maintain unit tests. We'll use them to write unit tests for the function above.

For this particular function, we'll want our unit tests to handle the following diverse scenarios (and under each scenario, we include a few examples as sub-bullets):
-"""
    prompt = full_code_explaination + prompt_to_explain_a_plan
    response = gpt35(prompt)
    return response, prompt


test_plan, prompt_to_get_test_plan = generate_a_test_plan(
    prompt_to_explain_code + code_explaination)
print(test_plan)


Valid input: the function should work correctly for a variety of input values
    - 0 seconds
    - 1 second
    - 60 seconds
    - 86400 seconds (one Earth day)
    - 88775.244 seconds (one Martian sol)
    - 1000000 seconds (about 11.3 Earth days)
    - 123456789 seconds (about 1422.6 Earth days)
- Invalid input: the function should raise an exception if the input is not a positive integer
    - -1 second
    - 3.14 seconds
    - "hello" (a string)
    - [1, 2, 3] (a list)
- Edge cases: the function should work correctly for edge cases that may not have been considered by the author
    - 88775.243 seconds (just under one Martian sol)
    - 88775.245 seconds (just over one Martian sol)
    - 86400.001 seconds (just over one Earth day)
    - 86399.999 seconds (just under one Earth day)
    - 88775.244 * 1000000 seconds (exactly 1000000 Martian sols)
    - 88775.244 * 123456789 seconds (exactly 123456789 Martian sols)


In [8]:
not_enough_test_plan = """The function is called with a valid number of seconds
    - `earth_to_mars_time(1)` should return `"1 Earth seconds is 0 Mars day(s), 0 hours, 0 minutes, 1 seconds on Mars."`
    - `earth_to_mars_time(59)` should return `"59 Earth seconds is 0 Mars day(s), 0 hours, 1 minutes, 0 seconds on Mars."`
    - `earth_to_mars_time(60)` should return `"60 Earth seconds is 0 Mars day(s), 0 hours, 1 minutes, 1 seconds on Mars."`
"""

approx_min_cases_to_cover = 7
elaboration_needed = test_plan.count("\n-") + 1 < approx_min_cases_to_cover
if elaboration_needed:
    prompt_to_elaborate_on_the_plan = f"""

In addition to the scenarios above, we'll also want to make sure we don't forget to test rare or unexpected edge cases (and under each edge case, we include a few examples as sub-bullets):
-"""
    more_test_plan, prompt_to_get_test_plan = generate_a_test_plan(
        prompt_to_explain_code + code_explaination + not_enough_test_plan + prompt_to_elaborate_on_the_plan)
    print(more_test_plan)


The function is called with a valid number of seconds
    - `earth_to_mars_time(1)` should return `"1 Earth seconds is 0 Mars day(s), 0 hours, 0 minutes, 1 seconds on Mars."`
    - `earth_to_mars_time(59)` should return `"59 Earth seconds is 0 Mars day(s), 0 hours, 1 minutes, 0 seconds on Mars."`
    - `earth_to_mars_time(60)` should return `"60 Earth seconds is 0 Mars day(s), 0 hours, 1 minutes, 1 seconds on Mars."`
- The function is called with a negative number of seconds
    - `earth_to_mars_time(-1)` should raise a `ValueError` exception
    - `earth_to_mars_time(-59)` should raise a `ValueError` exception
    - `earth_to_mars_time(-60)` should raise a `ValueError` exception
- The function is called with a non-integer number of seconds
    - `earth_to_mars_time(1.5)` should raise a `TypeError` exception
    - `earth_to_mars_time(59.9)` should raise a `TypeError` exception
    - `earth_to_mars_time(60.1)` should raise a `TypeError` exception
- The function is called with a very lar

In [9]:
def generate_test_cases(function_to_test, unit_test_package="unittest"):
    prompt_to_generate_the_unit_test = f"""

Before going into the individual tests, let's first look at the complete suite of unit tests as a cohesive whole. We've added helpful comments to explain what each line does.
```python
import {unit_test_package}  # used for our unit tests

{function_to_test}

"""
    full_unit_test_prompt = prompt_to_explain_code + \
        code_explaination + test_plan + prompt_to_generate_the_unit_test
    return gpt35(model="gpt-3.5-turbo-instruct", prompt=full_unit_test_prompt, stop="```"), prompt_to_generate_the_unit_test


unit_test_response, prompt_to_generate_the_unit_test = generate_test_cases(
    code)
print(unit_test_response)


class TestEarthToMarsTime(unittest.TestCase):
    """A suite of tests for the earth_to_mars_time function."""

    def test_valid_input(self):
        """Tests valid input values."""
        # Test 0 seconds
        self.assertEqual(earth_to_mars_time(0), (0, 0, 0, 0))

        # Test 1 second
        self.assertEqual(earth_to_mars_time(1), (0, 0, 0, 1))

        # Test 60 seconds
        self.assertEqual(earth_to_mars_time(60), (0, 0, 1, 0))

        # Test 86400 seconds (one Earth day)
        self.assertEqual(earth_to_mars_time(86400), (1, 0, 0, 0))

        # Test 88775.244 seconds (one Martian sol)
        self.assertEqual(earth_to_mars_time(88775.244), (1, 0, 0, 0))

        # Test 1000000 seconds (about 11.3 Earth days)
        self.assertEqual(earth_to_mars_time(1000000), (11, 8, 41, 20))

        # Test 123456789 seconds (about 1422.6 Earth days)
        self.assertEqual(earth_to_mars_time(123456789), (1422, 12, 41, 9))

    def test_invalid_input(self):
        """Tests inval

In [10]:
import ast

code_start_index = prompt_to_generate_the_unit_test.find(
    "```python\n") + len("```python\n")
code_output = prompt_to_generate_the_unit_test[code_start_index:] + \
    unit_test_response
try:
    ast.parse(code_output)
except SyntaxError as e:
    print(f"Syntax error in generated code: {e}")


the function takes in a number representing seconds on Earth.
- Next, it calculates the length of a Martian sol in Earth seconds. This value is stored in the variable `mars_sol_length`.
- Then, it converts the input `earth_seconds` to Mars seconds by multiplying it by the ratio of `mars_sol_length` to Earth's 24-hour day (86400 seconds).
- The remaining code calculates the components of Mars time (sols, hours, minutes, and seconds) by using integer division and the modulus operator.
- Finally, the function returns these components as a tuple.
The input is a positive integer
    - 1000
    - 86400
    - 1000000
- The input is a negative integer
    - -1000
    - -86400
    - -1000000
- The input is a float
    - 1000.5
    - 86400.25
    - 1000000.75
- The input is a string
    - "1000"
    - "86400"
    - "1000000"
- The input is a list
    - [1000]
    - [86400]
    - [1000000]
- The input is a tuple
    - (1000,)
    - (86400,)
    - (1000000,)
- The input is a dictionary
    - {"sec